## Model

In [46]:
from pathlib import Path
import pandas as pd

from flask import Flask 
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text
from sqlalchemy.sql import func


app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////' + str(Path.cwd() / 'movieDB.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False 
app.app_context().push()

db = SQLAlchemy(app)

# many to many relation
movie_actor_association = db.Table(
    'movie_actor_relation',
    db.Column('id',  db.Integer, primary_key=True, autoincrement=True),
    db.Column('movie_id', db.String(10), db.ForeignKey('movie_info.movie_id')),
    db.Column('actor_id', db.Integer, db.ForeignKey('actor_info.actor_id')),
    db.Column('relation_type', db.String(20))
)

class Movie(db.Model):
    __tablename__ = 'movie_info'

    movie_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    movie_name = db.Column(db.String(20))
    release_date = db.Column(db.String(15))
    country = db.Column(db.String(20))
    type = db.Column(db.String(10))
    year = db.Column(db.Integer)
    
    moviebox = db.relationship('MovieBox', backref='movie_info', uselist=False)
    actors = db.relationship('Actor', secondary=movie_actor_association, backref='movies')
    
    def __repr__(self):
        return f'<Movie {self.movie_name}>'
    
class Actor(db.Model):
    __tablename__ = 'actor_info'

    actor_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    actor_name = db.Column(db.String(20))
    gender = db.Column(db.String(2))
    country = db.Column(db.String(20))
    
    movies = db.relationship('Movie', secondary=movie_actor_association, back_populates='actors')
    
    
    def __repr__(self):
        return f'<Actor {self.actor_name}>'    
    
# one to one relation
class MovieBox(db.Model):
    __tablename__ = 'movie_box'
    movie_id = db.Column(db.String(10), db.ForeignKey('movie_info.movie_id'), unique=True, nullable=False, primary_key=True)
    box = db.Column(db.Float)

## Method
> basic opertions and CRUD

In [45]:
def table_dataframe(table_class):
    query = text(f"SELECT * FROM {table_class.__tablename__};")
    with db.engine.connect() as connection:
        result = connection.execute(query)
    return pd.DataFrame(result)

def select_to_dataframe(query):
    with db.engine.connect() as connection:
        result = connection.execute(text(query))
    return pd.DataFrame(result)

def find_or_add_actor(actor_name):
    # Check if the actor already exists in the database
    existing_actor = Actor.query.filter_by(actor_name=actor_name).first()

    if existing_actor:
        return existing_actor
    else:
        # Create a new actor and add it to the session
        new_actor = Actor(name=name)
        # session.add(new_actor)
        return new_actor

# Create Movie
def create_movie(movie_name, release_date, country, type, year, actors, moviebox):
    maxid = db.session.query(func.max(Movie.movie_id)).scalar()
    movie_id = str(int(maxid) + 1)
    movie = Movie(movie_id=movie_id, movie_name=movie_name, release_date=release_date, country=country, type=type, year=year)
    # movie.actors  = [find_or_add_actor(actor) for actor in actors]
    for actor in actors:
        movie.add_actor(find_or_add_actor(actor), relation_type='Lead')
        
    db.session.add(movie)
    db.session.commit()

new_movie = dict(
    movie_name='坚如磐石', 
    release_date='2013/10/1',
    country='中国', 
    type='动作', 
    year=2013, 
    actors=['张国立', '雷佳音', '于和伟'], 
    moviebox=15.5
)

new_movie = dict(
    movie_name='满江红', 
    release_date='2013/5/15',
    country='中国', 
    type='动作', 
    year=2013, 
    actors=['沈腾', '张译', '易烊千玺'], 
    moviebox=15.5
)
create_movie(**new_movie)    

ArgumentError: Error creating backref 'movies' on relationship 'Movie.actors': property of that name exists on mapper 'Mapper[Actor(actor_info)]'

<Actor 沈腾>

In [105]:
table_dataframe(Movie).head()

,movie_id,movie_name,release_date,country,type,year
0,1001,战狼2,2017/7/27,中国,战争,2017
1,1002,哪吒之魔童降世,2019/7/26,中国,动画,2019
2,1003,流浪地球,2019/2/5,中国,科幻,2019
3,1004,复仇者联盟4,2019/4/24,美国,科幻,2019
4,1005,红海行动,2018/2/16,中国,战争,2018


In [106]:
table_dataframe(MovieBox).head()

,movie_id,box
0,1001,56.84
1,1002,50.15
2,1003,46.86
3,1004,42.50
4,1005,36.50


In [107]:
db.session.query(Movie).first().moviebox.box

56.84

In [108]:
select_to_dataframe('SELECT * FROM movie_actor_relation;').head()

,id,movie_id,actor_id,relation_type
0,1,1001,2001,主演
1,2,1001,2001,导演
2,3,1002,2002,导演
3,4,1003,2001,主演
4,5,1003,2003,主演


In [121]:
Movie.query.filter_by(movie_id=1003).first().actors

[<Actor 吴京>, <Actor 屈楚萧>, <Actor 郭帆>]